Importing Libraries

In [95]:
import pandas as pd   
import time

from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline
from matplotlib import pyplot

from numpy import mean
import pandas as pd

import warnings 
warnings.filterwarnings('ignore')

Loading Dataset

In [96]:
train = pd.read_csv('Training.csv')
test = pd.read_csv('Testing.csv')
sample_data = pd.read_csv('Sample.csv')

In [97]:
# Deleted column row ID from df1
train.drop(['row ID'], axis=1, inplace=True)

Converting cateogry columns to string

In [98]:
# Changed WorkClass from int64 to str
train['WorkClass'] = train['WorkClass'].astype('str')
# Changed Occupation from int64 to str
train['Occupation'] = train['Occupation'].astype('str')
# Changed Education Level from int64 to str
train['Education Level'] = train['Education Level'].astype('str')
# Changed Marital Status from int64 to str
train['Marital Status'] = train['Marital Status'].astype('str')
# Changed Gender from int64 to str
train['Gender'] = train['Gender'].astype('str')
# Changed Native Country from int64 to str
train['Native Country'] = train['Native Country'].astype('str')
# Changed X3 from int64 to str
train['X3'] = train['X3'].astype('str')
# Deleted column row ID from test
test.drop(['row ID'], axis=1, inplace=True)
# Changed Native Country from int64 to str
test['Native Country'] = test['Native Country'].astype('str')
# Changed Gender from int64 to str
test['Gender'] = test['Gender'].astype('str')
# Changed X3 from int64 to str
test['X3'] = test['X3'].astype('str')
# Changed Occupation from int64 to str
test['Occupation'] = test['Occupation'].astype('str')
# Changed Marital Status from int64 to str
test['Marital Status'] = test['Marital Status'].astype('str')
# Changed Education Level from int64 to str
test['Education Level'] = test['Education Level'].astype('str')
# Changed WorkClass from int64 to str
test['WorkClass'] = test['WorkClass'].astype('str')

In [99]:
trainX = train.loc[:, train.columns != 'High Income']
trainY = train[['High Income']]

Merging the dataset for log transformation

In [100]:
print(trainX.shape, test.shape)
trainX.shape[0] + test.shape[0]
full_data = pd.concat([trainX, test], axis=0)
full_data.shape

(68378, 13) (29305, 13)


(97683, 13)

In [18]:
numeric_train_columns = pd.DataFrame(full_data.select_dtypes(np.number).columns, columns=['Feature'])
numeric_train_columns['Skew'] = numeric_train_columns['Feature'].apply(lambda feature: scipy.stats.skew(full_data[feature]))
numeric_train_columns['Abs Skew'] = numeric_train_columns['Skew'].apply(abs)
numeric_train_columns['Skewed'] = numeric_train_columns['Abs Skew'].apply(lambda x: True if x > 0.5 else False)
numeric_train_columns

,Feature,Skew,Abs Skew,Skewed
0,Age,0.749752,0.749752,True
1,X1,1.673820,1.673820,True
2,X2,-0.596001,0.596001,True
3,X4,1.669497,1.669497,True
4,X5,1.554276,1.554276,True
5,Hours Per Week Working,0.692582,0.692582,True


In [19]:
for column in numeric_train_columns.query('Skewed == True')['Feature'].values:
    full_data[column] = np.log1p(full_data[column])

In [20]:
numeric_train_columns2 = pd.DataFrame(full_data.select_dtypes(np.number).columns, columns=['Feature'])
numeric_train_columns2['Skew'] = numeric_train_columns2['Feature'].apply(lambda feature: scipy.stats.skew(full_data[feature]))
numeric_train_columns2['Abs Skew'] = numeric_train_columns2['Skew'].apply(abs)  
numeric_train_columns2['Skewed'] = numeric_train_columns2['Abs Skew'].apply(lambda x: True if x > 0.5 else False) 
numeric_train_columns2

,Feature,Skew,Abs Skew,Skewed
0,Age,-0.030965,0.030965,False
1,X1,0.165746,0.165746,False
2,X2,-1.734169,1.734169,True
3,X4,0.305614,0.305614,False
4,X5,0.770851,0.770851,True
5,Hours Per Week Working,-1.816578,1.816578,True


In [21]:
full_data.head(3)

,Age,WorkClass,X1,Education Level,X2,Marital Status,Occupation,X3,Gender,X4,X5,Hours Per Week Working,Native Country
0,3.663562,2,12.281398,1,2.302585,2,2,0,0,0.000000,0.000000,3.713572,0
1,3.988984,2,12.366157,2,2.079442,1,2,1,0,0.000000,0.000000,3.713572,0
2,3.637586,2,12.558780,3,2.708050,1,1,0,1,0.000000,0.000000,3.713572,0
3,3.970292,1,12.253161,1,2.302585,1,1,0,0,0.000000,0.000000,3.828641,0
4,3.465736,2,10.731646,3,2.708050,0,3,0,1,9.552866,0.000000,3.931826,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29300,3.663562,2,10.401745,7,2.484907,1,6,0,0,0.000000,0.000000,4.025352,0
29301,3.961461,2,10.668175,3,2.673935,1,3,0,1,0.000000,0.000000,3.467475,0
29302,3.919528,2,11.826064,3,2.500297,1,3,0,1,0.000000,0.000000,3.581959,0
29303,4.060443,4,11.612028,1,2.302585,1,6,0,0,11.512925,0.000000,3.713572,0


In [101]:
#### Train and test split
train_new = full_data.iloc[:train.shape[0]]
test_new = full_data.iloc[train.shape[0]:]

In [102]:
train_onehot = pd.get_dummies(train_new, drop_first=True)
test_onehot = pd.get_dummies(test_new, drop_first=True)

In [89]:
rf = GradientBoostingClassifier(max_depth=25,n_estimators=1500)
rf.fit(train_onehot, trainY)
md_probs = rf.predict_proba(test_onehot)

RepeatedKFold for checking the score of model on train dataset

In [ ]:
def fitting_models_CV2():
    cv = RepeatedKFold(n_splits=10, n_repeats=1)#, random_state=1)
    
    lr=LogisticRegression()
    dt = DecisionTreeClassifier(max_depth=500)    
    rf = RandomForestClassifier(max_depth=200,n_estimators=1600)
    gb = GradientBoostingClassifier(max_depth=50,n_estimators=150)
    nb_g = GaussianNB()
    nb_c = CategoricalNB()

    kn = KNeighborsClassifier(n_neighbors=200)
    pipe_kn = Pipeline([("scaler", MinMaxScaler()), 
                         ("knr", KNeighborsClassifier(n_neighbors=500))])
    clfs = [
        ('Logistic Regression', lr),        
        ('Decision Tree', dt),       
        ('Random Forest', rf),
        ('Gradient Boosting', gb),
        ('KNearest Neighbor',kn),
        ('Scaled KNearest',pipe_kn)  
        ('Naive bayes gaussian', nb_g) 

    ]
    for name,clf in clfs:
        start = time.perf_counter()
        #scores = cross_val_score(clf, X, y, scoring="accuracy", cv=cv) 
        scores = cross_val_score(clf, train_onehot, trainY, scoring="roc_auc", cv=cv) 
        end = time.perf_counter()        
        score = format(mean(scores), '.4f')
        duration = format((end-start),'.4f')
        print("{} : {} - {}".format(name,score,duration))

fitting_models_CV2()

In [110]:
model = RandomForestClassifier(max_depth=800,n_estimators=1600)
model.fit(train_onehot,trainY)
md_probs = model.predict_proba(test_onehot)

In [105]:
row_id = sample_data['row ID']

In [106]:
col1 = list(row_id)
col2 = list(md_probs[:,1])

In [107]:
final_data = pd.DataFrame({'row ID': row_id, 'High Income': col2})

In [108]:
final_data.to_csv('output.csv', index=False)

In [109]:
final_data

,row ID,High Income
0,Row0,0.509695
1,Row1,0.520756
2,Row4,0.320411
3,Row6,0.013256
4,Row10,0.633935
...,...,...
29300,Row32506dupl_65093,0.423924
29301,Row32513dupl_65096,0.646065
29302,Row32513dupl_65097,0.638752
29303,Row32518dupl_65098,0.969782
